In [1]:
#read data(exclude replies) 
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk import SnowballStemmer
import pandas as pd

text = pd.read_csv('Jan6(excludes replies).csv')
print('The number of tweets (excludes replies):',len(text))
#Text cleaning
import gensim
import gensim.corpora as corpora
import re
stemmer = SnowballStemmer('english')
porter = PorterStemmer()
lemmatizer = WordNetLemmatizer()
stop_words = stopwords.words('english')
stop_words += ['cultured','clean','cultivated','meat']
b = []
for i,u in text.iterrows():
    a = []
    word =''
    for words in str(u['tweets']).split():
        if '@' not in words: #remove @users
            if '#' not in words:
                if 'http' not in words: #remove URLs
                    if'&amp' not in words: #remove symbol
                        words = words.lower()# lower form
                        words = re.sub(r'[^a-zA-Z]', ' ', words) #replace non-alphabets characters with space. From "can't" to "can t"
                        if len(words)>3:
                            word += (words+' ')
    doc = ''
    for token in word.split():
        if token not in stop_words:
            token = porter.stem(token) #root form
            doc += (token+' ')
    b.append(doc)
text['processed']=[i for i in b]

The number of tweets (excludes replies): 6501


In [21]:
#unigram
unigram = []
unigram_list = []
for index, i in text.iterrows():
    unigram=''
    for word in i['processed'].split():
        unigram+= word+' '
    unigram_list.append(unigram)


import guidedlda
import numpy as np
import gensim

seed_topic_list = [['environ','environment','carbon','climat','greenhous','emiss'], #Environmental Impact
                   ['plant','vegan','vegetarian','animal','without','slaughter',
                   'cruelti','welfar'],# Animal Welfare
                  ['antibiot','antimicrobi','resist','safeti','contamin','consumpt','popul',
                  'demand','secur','wast','industri','suppli','convent','regular','system','market','consum','store','groceri'], #Food System
                  ['seafood','sea','ocean','fish','aquacultur','shrimp'],#Seafood
                  ['tast','tender','textur','juici','test','chicken','duck','poultri','egg'],#Poultry
                  ['seri','fund','funder','rais','invest','investor','dollar','pound','announc', 
                  'happi','thrill','excit'], #Announcement
                  ['thank','thankyou','support','shoutout','help'], #Appreciation
                  ['confer','regist','live','symposium','stream','livestream','summit'], #conference and summit
                  ['check','post','paper','interview','articl','blog','news','break'], #Media
                  ['hire','join','team']]#Hiring Information

In [27]:
from sklearn.feature_extraction.text import CountVectorizer
model = guidedlda.GuidedLDA(n_topics=10,n_iter=200,random_state=1,refresh=10,alpha=0.1,eta=0.01)
vectorizer = CountVectorizer(min_df = 2)
X = vectorizer.fit_transform(text['processed'])

vocab = vectorizer.get_feature_names()
word2id = dict((v,idx) for idx,v in enumerate(vocab))
seed_topics = {}
for t_id, st in enumerate(seed_topic_list):
    for word in st:
        seed_topics[word2id[word]] = t_id

model.fit(X.toarray(),seed_topics=seed_topics,seed_confidence=0.35)
topic_word = model.topic_word_
n_top_words = 30
vocab = tuple(vocab)

for i, topic_dist in enumerate(topic_word):
    print('\n')
    print('Topic:',i)
    words_probability = np.array(-topic_dist)
    for index in range(n_top_words):
        print(round(abs(np.sort(words_probability))[:(n_top_words)][index],4),'*',
              np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1][index],sep='',end='  ')
print('\n')    
X = X.toarray()
doc_topic = model.transform(X)
topic_number = []
number = []
for i in range(len(doc_topic)-1):
    topic_number.append(doc_topic[i].argmax())
    number.append('1')
data = pd.DataFrame(data=[i for i in topic_number],columns=['topic_number'])
data['number'] = [i for i in number]
print(data.groupby('topic_number')['number'].count())

INFO:guidedlda:n_documents: 6501
INFO:guidedlda:vocab_size: 3621
INFO:guidedlda:n_words: 60068
INFO:guidedlda:n_topics: 10
INFO:guidedlda:n_iter: 200
INFO:guidedlda:<0> log likelihood: -677657
INFO:guidedlda:<10> log likelihood: -488456
INFO:guidedlda:<20> log likelihood: -472934
INFO:guidedlda:<30> log likelihood: -466380
INFO:guidedlda:<40> log likelihood: -462785
INFO:guidedlda:<50> log likelihood: -460274
INFO:guidedlda:<60> log likelihood: -458965
INFO:guidedlda:<70> log likelihood: -458062
INFO:guidedlda:<80> log likelihood: -456967
INFO:guidedlda:<90> log likelihood: -456704
INFO:guidedlda:<100> log likelihood: -455979
INFO:guidedlda:<110> log likelihood: -455437
INFO:guidedlda:<120> log likelihood: -455343
INFO:guidedlda:<130> log likelihood: -454906
INFO:guidedlda:<140> log likelihood: -454991
INFO:guidedlda:<150> log likelihood: -454700
INFO:guidedlda:<160> log likelihood: -454300
INFO:guidedlda:<170> log likelihood: -454075
INFO:guidedlda:<180> log likelihood: -454155
INFO:g



Topic: 0
0.0255*chang  0.0255*food  0.0243*anim  0.0194*world  0.0182*sustain  0.0159*climat  0.0145*human  0.0117*need  0.0114*feed  0.0101*health  0.0101*global  0.01*planet  0.0097*environment  0.0096*solut  0.0088*grow  0.0088*peopl  0.0088*agricultur  0.0083*system  0.0081*product  0.0077*reduc  0.0077*impact  0.0074*industri  0.0072*save  0.0071*help  0.0068*environ  0.0068*earth  0.0068*eat  0.0068*produc  0.0067*use  0.0065*could  

Topic: 1
0.0571*anim  0.026*without  0.0255*make  0.0216*cell  0.0213*base  0.0176*free  0.0152*real  0.0152*plant  0.0147*grow  0.0142*slaughter  0.0135*food  0.0111*want  0.0111*futur  0.0106*burger  0.0103*need  0.0098*tast  0.0096*produc  0.0096*eat  0.0093*peopl  0.0086*kill  0.0084*know  0.0083*help  0.0078*harm  0.0078*like  0.0078*vegan  0.0071*answer  0.0071*made  0.0068*cow  0.0064*better  0.0062*would  

Topic: 2
0.0285*product  0.0277*base  0.0209*market  0.0203*industri  0.0185*year  0.0183*cell  0.0176*plant  0.0135*seafood  0.0127*f

0.0098*tomorrow  0.0098*innov  0.0092*talk  0.0089*summit  0.0085*world  0.0085*present  0.0083*ceo  0.0079*meet  0.0077*cellular  0.0076*altern  0.0074*technolog  0.007*tune  0.0067*sign  0.0067*come  

Topic: 8
0.0228*founder  0.0225*featur  0.0176*co  0.0173*meat  0.0161*thank  0.0157*cell  0.0154*check  0.0154*compani  0.0152*interview  0.014*post  0.0137*read  0.0128*latest  0.0112*base  0.0105*great  0.0102*podcast  0.01*shiok  0.01*stori  0.0098*talk  0.0093*discuss  0.0092*watch  0.0088*innov  0.0086*list  0.0085*startup  0.0081*seafood  0.0079*food  0.0078*industri  0.0076*articl  0.0074*today  0.0074*video  0.0074*news  

Topic: 9
0.0321*team  0.0226*join  0.0204*food  0.0196*meat  0.0194*work  0.0188*excit  0.0151*make  0.0143*research  0.0136*year  0.0127*help  0.0125*look  0.0124*futur  0.0116*memphi  0.0105*sustain  0.0096*share  0.0094*support  0.0094*welcom  0.0093*hire  0.0089*scientist  0.0088*develop  0.0086*pleas  0.0083*proud  0.0082*happi  0.008*world  0.0074*appl